# Data Structures for Search

Contents:

- B-Trees for range searches of numerical and date values-
- Inverted Index for searching relevant documents based on text.
- Doc Values (Columnar data representatios) for fast field/column-wise aggregation operations. 

## B-Tree

See [`README.md`](./README.md) and [`../README.md`](../README.md).

Let's consider with have the following set of documents:

```
[
  { "id": 1, "name": "Product A", "price": 10 },
  { "id": 2, "name": "Product B", "price": 20 },
  { "id": 3, "name": "Product C", "price": 15 },
  { "id": 4, "name": "Product D", "price": 25 },
  { "id": 5, "name": "Product E", "price": 30 }
]
```

We want to build a B-Tree for them to be able to perform range-based search and filtering.
We want to store the document ids in the nodes, but all tree operations are done based on the price value of the document.

In [5]:
class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Minimum degree (defines the range for number of keys)
        self.leaf = leaf
        self.keys = []  # Array of keys
        self.values = []  # Array of values (document IDs)
        self.children = []  # Array of child pointers

    def insert_non_full(self, key, value):
        i = len(self.keys) - 1

        if self.leaf:
            self.keys.append(0)
            self.values.append(0)
            while i >= 0 and self.keys[i] > key:
                self.keys[i + 1] = self.keys[i]
                self.values[i + 1] = self.values[i]
                i -= 1
            self.keys[i + 1] = key
            self.values[i + 1] = value
        else:
            while i >= 0 and self.keys[i] > key:
                i -= 1
            if len(self.children[i + 1].keys) == 2 * self.t - 1:
                self.split_child(i + 1, self.children[i + 1])
                if self.keys[i + 1] < key:
                    i += 1
            self.children[i + 1].insert_non_full(key, value)

    def split_child(self, i, y):
        t = self.t
        z = BTreeNode(t, y.leaf)
        self.children.insert(i + 1, z)
        self.keys.insert(i, y.keys[t - 1])
        self.values.insert(i, y.values[t - 1])
        z.keys = y.keys[t:(2 * t - 1)]
        z.values = y.values[t:(2 * t - 1)]
        y.keys = y.keys[0:(t - 1)]
        y.values = y.values[0:(t - 1)]

        if not y.leaf:
            z.children = y.children[t:(2 * t)]
            y.children = y.children[0:t]

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, True)
        self.t = t

    def insert(self, key, value):
        root = self.root
        if len(root.keys) == 2 * self.t - 1:
            temp = BTreeNode(self.t, False)
            temp.children.insert(0, root)
            temp.split_child(0, root)
            i = 0
            if temp.keys[0] < key:
                i += 1
            temp.children[i].insert_non_full(key, value)
            self.root = temp
        else:
            root.insert_non_full(key, value)

    def print_tree(self, node, lvl=0):
        print("Level", lvl, ":", list(zip(node.keys, node.values)))
        lvl += 1
        if len(node.children) > 0:
            for child in node.children:
                self.print_tree(child, lvl)

In [6]:
# Example usage
btree = BTree(2)  # B-tree with minimum degree 2
documents = [
    {"id": 1, "price": 10},
    {"id": 2, "price": 20},
    {"id": 3, "price": 15},
    {"id": 4, "price": 25},
    {"id": 5, "price": 30}
]

for doc in documents:
    btree.insert(doc["price"], doc["id"])

btree.print_tree(btree.root)


Level 0 : [(15, 3)]
Level 1 : [(10, 1)]
Level 1 : [(20, 2), (25, 4), (30, 5)]


## Inverted Index

See [`README.md`](./README.md) and [`../README.md`](../README.md).

Let's consider with have the following set of documents (same as before):

```
[
  { "id": 1, "name": "Product A", "price": 10 },
  { "id": 2, "name": "Product B", "price": 20 },
  { "id": 3, "name": "Product C", "price": 15 },
  { "id": 4, "name": "Product D", "price": 25 },
  { "id": 5, "name": "Product E", "price": 30 }
]
```

We want to build an inverted index for the field `name`.

In [7]:
documents = [
  { "id": 1, "name": "Product A", "price": 10 },
  { "id": 2, "name": "Product B", "price": 20 },
  { "id": 3, "name": "Product C", "price": 15 },
  { "id": 4, "name": "Product D", "price": 25 },
  { "id": 5, "name": "Product E", "price": 30 }
]

In [8]:
### --- Tokenization
tokenized_docs = []
for doc in documents:
    tokens = doc['name'].split()
    tokenized_docs.append((doc['id'], tokens))

print(tokenized_docs)
# Output: [(1, ['Product', 'A']), (2, ['Product', 'B']), (3, ['Product', 'C', 'Product', 'A'])]

[(1, ['Product', 'A']), (2, ['Product', 'B']), (3, ['Product', 'C']), (4, ['Product', 'D']), (5, ['Product', 'E'])]


In [9]:
### -- Compute Term Frequencies (TF)
from collections import defaultdict

tf = defaultdict(lambda: defaultdict(int))
for doc_id, tokens in tokenized_docs:
    for token in tokens:
        tf[doc_id][token.lower()] += 1

print(dict(tf))
# Output: {1: {'product': 1, 'a': 1}, 2: {'product': 1, 'b': 1}, 3: {'product': 2, 'c': 1, 'a': 1}}

{1: defaultdict(<class 'int'>, {'product': 1, 'a': 1}), 2: defaultdict(<class 'int'>, {'product': 1, 'b': 1}), 3: defaultdict(<class 'int'>, {'product': 1, 'c': 1}), 4: defaultdict(<class 'int'>, {'product': 1, 'd': 1}), 5: defaultdict(<class 'int'>, {'product': 1, 'e': 1})}


In [10]:
### -- Compute Inverse Document Frequencies (IDF)
import math

df = defaultdict(int)
for doc_id, tokens in tokenized_docs:
    for token in set(tokens):
        df[token.lower()] += 1

idf = {term: math.log(len(documents) / df[term]) for term in df}

print(idf)
# Output: {'product': 0.0, 'a': 0.4054651081081644, 'b': 1.0986122886681098, 'c': 1.0986122886681098}


{'product': 0.0, 'a': 1.6094379124341003, 'b': 1.6094379124341003, 'c': 1.6094379124341003, 'd': 1.6094379124341003, 'e': 1.6094379124341003}


In [11]:
### -- Compute TF-IDF
tf_idf = defaultdict(lambda: defaultdict(float))
for doc_id, tokens in tokenized_docs:
    for token in tokens:
        tf_idf[doc_id][token.lower()] = tf[doc_id][token.lower()] * idf[token.lower()]

print(dict(tf_idf))
# Output: {1: {'product': 0.0, 'a': 0.4054651081081644}, 2: {'product': 0.0, 'b': 1.0986122886681098}, 3: {'product': 0.0, 'c': 1.0986122886681098, 'a': 0.4054651081081644}}


{1: defaultdict(<class 'float'>, {'product': 0.0, 'a': 1.6094379124341003}), 2: defaultdict(<class 'float'>, {'product': 0.0, 'b': 1.6094379124341003}), 3: defaultdict(<class 'float'>, {'product': 0.0, 'c': 1.6094379124341003}), 4: defaultdict(<class 'float'>, {'product': 0.0, 'd': 1.6094379124341003}), 5: defaultdict(<class 'float'>, {'product': 0.0, 'e': 1.6094379124341003})}


In [12]:
### -- Build the Inverted Index
inverted_index = defaultdict(list)
for doc_id, scores in tf_idf.items():
    for term, score in scores.items():
        inverted_index[term].append((doc_id, score))

print(dict(inverted_index))
# Output: {'product': [(1, 0.0), (2, 0.0), (3, 0.0)], 'a': [(1, 0.4054651081081644), (3, 0.4054651081081644)], 'b': [(2, 1.0986122886681098)], 'c': [(3, 1.0986122886681098)]}


{'product': [(1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)], 'a': [(1, 1.6094379124341003)], 'b': [(2, 1.6094379124341003)], 'c': [(3, 1.6094379124341003)], 'd': [(4, 1.6094379124341003)], 'e': [(5, 1.6094379124341003)]}


## Doc Values, Columnar Data

See [`README.md`](./README.md) and [`../README.md`](../README.md).

Let's consider with have the following set of documents (same as before):

```
[
  { "id": 1, "name": "Product A", "price": 10 },
  { "id": 2, "name": "Product B", "price": 20 },
  { "id": 3, "name": "Product C", "price": 15 },
  { "id": 4, "name": "Product D", "price": 25 },
  { "id": 5, "name": "Product E", "price": 30 }
]
```

We want to build a Doc Value or columnar representation of `price` to be able to compute fast field aggragation operations.

In [1]:
documents = [
  { "id": 1, "name": "Product A", "price": 10 },
  { "id": 2, "name": "Product B", "price": 20 },
  { "id": 3, "name": "Product C", "price": 15 },
  { "id": 4, "name": "Product D", "price": 25 },
  { "id": 5, "name": "Product E", "price": 30 }
]

In [2]:
import numpy as np

# Initialize numpy array for prices
prices = np.zeros(len(documents))

# Initialize bitstring
bitstring = np.zeros(len(documents), dtype=int)

# Initialize dictionary to map document IDs to array indices
doc_id_to_index = {}


In [3]:
for idx, doc in enumerate(documents):
    prices[idx] = doc["price"]
    bitstring[idx] = 1  # 1 indicates the presence of the document
    doc_id_to_index[doc["id"]] = idx

print("Prices Array:", prices)
print("Bitstring:", bitstring)
print("Document ID to Index Mapping:", doc_id_to_index)

Prices Array: [10. 20. 15. 25. 30.]
Bitstring: [1 1 1 1 1]
Document ID to Index Mapping: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4}


In [10]:
# Switch off documents with IDs 1 and 3
bitstring[doc_id_to_index[1]] = 0
bitstring[doc_id_to_index[3]] = 0

print("Updated Bitstring:", bitstring)

# Compute the mean price using masked array
mean_price = np.ma.masked_array(prices, mask=bitstring == 0).mean()

# Alternative
# masked_prices = prices[bitstring == 1]
# mean_price = masked_prices.mean()
# Which option is faster and more efficient?
# The new array will require more memory,
# the speed is benchmarked below...

print("Mean Price:", mean_price)

Updated Bitstring: [0 1 0 ... 1 1 1]
Mean Price: 49.99163888795988


### Benchmarking of Masked vs. New Array Aggregation Operations

In [11]:
import time

# Large example dataset
num_docs = 10_000_000
documents = [{"id": i, "price": np.random.randint(1, 100)} for i in range(num_docs)]

# Initialize numpy array for prices
prices = np.zeros(num_docs)

# Initialize bitstring
bitstring = np.ones(num_docs, dtype=int)

# Populate the arrays and mappings
for idx, doc in enumerate(documents):
    prices[idx] = doc["price"]

# Switch off some documents
bitstring[::10] = 0  # Switch off every 10th document

# Benchmark creating a new array
start_time = time.time()
masked_prices = prices[bitstring == 1]
mean_price_new_array = masked_prices.mean()
new_array_time = time.time() - start_time

# Benchmark applying a mask
start_time = time.time()
mean_price_mask = np.ma.masked_array(prices, mask=bitstring == 0).mean()
mask_time = time.time() - start_time

print(f"New array mean price: {mean_price_new_array}, Time: {new_array_time} seconds")
print(f"Masked array mean price: {mean_price_mask}, Time: {mask_time} seconds")
# The new array is almost 2x faster than the masked array approach
# but that's maybe not always the case...

New array mean price: 50.00747511111111, Time: 0.057581424713134766 seconds
Masked array mean price: 50.00747511111111, Time: 0.04833984375 seconds
